In [ ]:
import sys
import pandas as pd

import numpy as np
import skopt

In [ ]:
training=pd.read_csv("/path_to_ADNI_training_dataset.csv")
test=pd.read_csv("/path_to_ADNI_test_dataset.csv")

In [ ]:
datasetAIBL=pd.read_csv("/path_to_AIBL_dataset.csv")
datasetOASIS=pd.read_csv("/path_to_OASIS_dataset.csv")

In [ ]:
training.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
test.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetAIBL.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetOASIS.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)

In [ ]:
training=training.replace([np.inf, -np.inf], np.nan)
test=test.replace([np.inf, -np.inf], np.nan)
datasetAIBL=datasetAIBL.replace([np.inf, -np.inf], np.nan)
datasetOASIS=datasetOASIS.replace([np.inf, -np.inf], np.nan)

In [ ]:
training=training.dropna(axis=1, how='all')

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import joblib

In [ ]:
#TODO CHANGE IF NESSESARY
featureSetSelected="MRI+SocioDemography"
model="svmPoly"
selectMethod="yes"

In [ ]:
clf = joblib.load('/path_to_save_model/model_'+featureSetSelected+"_"+model+"_"+selectMethod+'.pkl') 


import shap



dataTrain=training.filter(clf.best_estimator_.named_steps["preprocessing"].transformers_[0][2],axis=1)
dataGes=dataGes.append(datasetAIBL, ignore_index=True)
dataGes=dataGes.append(datasetOASIS, ignore_index=True)
dataTestTrain=dataGes.filter(clf.best_estimator_.named_steps["preprocessing"].transformers_[0][2],axis=1)




np.random.seed(312)
def predictTest(Xtrain):
    XtrainPD= pd.DataFrame(Xtrain, columns =dataTrain.columns)
    return(clf.predict_proba(XtrainPD)[:,1])
# use Kernel SHAP to explain test set predictions
explainer = shap.KernelExplainer(predictTest, dataTrain)#, link="logit")
shap_values = explainer.shap_values(dataTestTrain, nsamples=1000)

import matplotlib.pyplot as plt
shap.summary_plot(shap_values,dataTestTrain,max_display=10,show=False)
plt.savefig('/path_to_save_figures/shap_plot_'+featureSetSelected+"_"+model+"_"+selectMethod+'.pdf',bbox_inches='tight')

import joblib
joblib.dump(shap_values, '/path_to_save_shap_values/shap_values'+featureSetSelected+"_"+model+"_"+selectMethod+'.pkl', compress = 1)